In [ ]:

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/עותק של Supplementary data - responses and measures (1).csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Remove punctuation, numbers, and special characters.
Convert text to lowercase.
Tokenize the paragraphs into words.

In [ ]:

all = (df[df['Condition'].isin({'Human', '2','1','AI'})])
# set df['Condition'].isin({'Human', '2'} to 2 and df['Condition'].isin({'AI', '1'} to 1
all['Condition'] = all['Condition'].apply(lambda x: 2 if x in {'Human', '2'} else 1)

/tmp/ipython-input-1884332789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all['Condition'] = all['Condition'].apply(lambda x: 2 if x in {'Human', '2'} else 1)


In [ ]:

hrel = all[['Response', 'EmpathyQ_1','Condition']]

# Rename columns to match script expectations
hrel = hrel.rename(columns={"Response": "response", "EmpathyQ_1": "label",'Condition':'condition'})

hrel = hrel.dropna(subset=["response", "label"])  # Drop rows with NaN
hrel = hrel[hrel["response"].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Keep valid strings
hrel["label"] = hrel["label"].astype(float)  # Ensure empathy is float


In [ ]:
%%writefile ex1.py
import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import wandb
import torch
import torch.nn as nn
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def parse_args():
    parser = argparse.ArgumentParser(description="Fine-tune models for binary empathy classification with weighted loss")
    parser.add_argument("--max_train_samples", type=int, default=-1, help="Number of training samples or -1 for all")
    parser.add_argument("--max_predict_samples", type=int, default=-1, help="Number of test samples or -1 for all")
    parser.add_argument("--num_train_epochs", type=int, default=3, help="Number of training epochs")
    parser.add_argument("--lr", type=float, default=2e-5, help="Learning rate")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for training and evaluation")
    parser.add_argument("--do_train", action="store_true", help="Run training")
    parser.add_argument("--do_predict", action="store_true", help="Run prediction")
    parser.add_argument("--model_path", type=str, default="./results", help="Path to save/load model")
    parser.add_argument("--model_name", type=str, default="bert-base-uncased", help="Model name (bert-base-uncased, roberta-base, distilbert-base-uncased, electra-small-discriminator)")
    return parser.parse_args()

def compute_sample_weights(labels):
    """Compute sample weights for binary classification based on class frequency"""
    bins = np.array([0, 1, 2])  # Bins for classes 0 and 1
    freq, _ = np.histogram(labels, bins=bins)
    freq = freq + 1e-6  # Avoid division by zero
    weights = np.max(freq) / freq * 2.0
    sample_weights = np.array([weights[int(label)] for label in labels])
    logger.info(f"Sample weights for classes [0, 1]: {weights}")
    return sample_weights

def undersample_data(df, high_label=1, low_label=0, ratio=1.5):
    """Undersample high-empathy class to achieve ~2:1 ratio with low-empathy class"""
    high_df = df[df['labels'] == high_label]
    low_df = df[df['labels'] == low_label]
    n_low = len(low_df)
    n_high = min(len(high_df), n_low * ratio)
    high_df = high_df.sample(n=n_high, random_state=42)
    balanced_df = pd.concat([high_df, low_df]).sample(frac=1, random_state=42).reset_index(drop=True)
    logger.info(f"After undersampling: {len(high_df)} high, {len(low_df)} low samples")
    return balanced_df

def plot_confusion_matrix(labels, predictions, phase="test", model_name="model"):
    """Plot and save confusion matrix"""
    cm = confusion_matrix(labels, predictions, labels=[0, 1])
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Low", "High"], yticklabels=["Low", "High"])
    plt.title(f"Confusion Matrix ({phase.capitalize()}, {model_name})")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    output_path = os.path.join("results", f"confusion_matrix_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"Confusion matrix saved to {output_path}")
    wandb.log({f"confusion_matrix_{phase}_{model_name}": wandb.Image(output_path)})
    return cm

def plot_f1_scores(labels, predictions, phase="test", model_name="model"):
    """Plot and save per-class F1 scores"""
    f1 = f1_score(labels, predictions, labels=[0, 1], average=None, zero_division=0)
    plt.figure(figsize=(6, 4))
    plt.bar(["Low", "High"], f1, color="skyblue")
    plt.title(f"F1 Scores per Class ({phase.capitalize()}, {model_name})")
    plt.xlabel("Class")
    plt.ylabel("F1 Score")
    plt.ylim(0, 1)
    output_path = os.path.join("results", f"f1_scores_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"F1 scores plot saved to {output_path}")
    wandb.log({f"f1_scores_{phase}_{model_name}": wandb.Image(output_path)})
    return f1

def load_and_prepare_data(args, df):
    """Load and preprocess data for binary classification"""
    df = df.dropna(subset=['response', 'label', 'condition'])
    df['label'] = df['label'].astype(float)
    if not ((df['label'] >= 0) & (df['label'] <= 9)).all():
        logger.warning("Some labels are outside [0, 9]. Clipping may affect results.")
    df['labels'] = (df['label'] >= 6).astype(int)
    logger.info(f"Label distribution (binary): {df['labels'].value_counts().to_string()}")
    wandb.log({"label_distribution": wandb.Histogram(df['labels'])})

    if len(df) < 100:
        logger.warning("Dataset is small (<100 samples). Consider adding more samples.")

    sample_weights = compute_sample_weights(df['labels'].values)
    df['weight'] = sample_weights

    df = undersample_data(df, high_label=1, low_label=0, ratio=2)

    df['input_text'] = df.apply(lambda x: f"condition_{int(x['condition'])}: {x['response']}", axis=1)

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    logger.info(f"Train size: {len(train_df)}, Test size: {len(test_df)}")

    train_dataset = Dataset.from_pandas(train_df[['input_text', 'labels', 'weight']])
    test_dataset = Dataset.from_pandas(test_df[['input_text', 'labels']])

    try:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    except Exception as e:
        logger.error(f"Failed to load tokenizer for {args.model_name}: {e}")
        raise

    def tokenize_function(examples):
        return tokenizer(
            examples["input_text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    if args.max_train_samples != -1:
        train_dataset = train_dataset.select(range(min(args.max_train_samples, len(train_dataset))))
    if args.max_predict_samples != -1:
        test_dataset = test_dataset.select(range(min(args.max_predict_samples, len(test_dataset))))

    train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "weight"])
    test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    return train_dataset, test_dataset, tokenizer, 1

def compute_metrics(eval_pred):
    """Compute accuracy, F1-score, and confusion matrix"""
    logits, labels = eval_pred
    logits = logits.squeeze()
    predictions = (logits > 0).astype(int)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    f1_per_class = f1_score(labels, predictions, labels=[0, 1], average=None, zero_division=0)
    cm = plot_confusion_matrix(labels, predictions, phase="test", model_name=args.model_name.split("/")[-1])
    f1_per_class_dict = {f"f1_class_{i}": float(f1_per_class[i]) for i in range(2)}
    metrics = {
        "accuracy": float(acc),
        "f1_macro": float(f1),
        **f1_per_class_dict
    }
    wandb.log(metrics)
    return metrics

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """Compute weighted BCE loss"""
        logger.debug(f"Inputs keys: {list(inputs.keys())}")
        labels = inputs.pop("labels")
        weights = inputs.pop("weight", torch.ones_like(labels))
        outputs = model(**inputs)
        logits = outputs.logits
        # Ensure logits and labels have compatible shapes
        if logits.dim() > 1:
            logits = logits.squeeze(-1)  # Squeeze only the last dimension if needed
        if logits.shape != labels.shape:
            logger.debug(f"Logits shape: {logits.shape}, Labels shape: {labels.shape}")
            logits = logits.view(-1)  # Flatten to [batch_size]
        loss_fn = nn.BCEWithLogitsLoss(reduction='none')
        loss = loss_fn(logits, labels.float())
        weighted_loss = torch.mean(weights * loss)
        return (weighted_loss, outputs) if return_outputs else weighted_loss

def train_model(args, train_dataset, test_dataset, num_labels):
    """Train the model with weighted loss"""
    os.makedirs(args.model_path, exist_ok=True)
    try:
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=num_labels)
    except Exception as e:
        logger.error(f"Failed to load model {args.model_name}: {e}")
        raise

    training_args = TrainingArguments(
        output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
        eval_strategy="no",
        save_strategy="epoch",
        learning_rate=args.lr,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.num_train_epochs,
        weight_decay=0.01,
        logging_steps=10,
        save_total_limit=2,
        report_to="wandb",
        run_name=f"{args.model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}",
        fp16=True,
        dataloader_drop_last=True  # Drop incomplete last batch
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=None,
        compute_metrics=None
    )

    try:
        trainer.train()
    except Exception as e:
        logger.error(f"Training failed for {args.model_name}: {e}")
        raise

    trainer.save_model(os.path.join(args.model_path, args.model_name.split("/")[-1]))
    return trainer

def predict(trainer, test_dataset, args):
    """Make predictions and compute metrics"""
    try:
        predictions = trainer.predict(test_dataset)
        logits = predictions.predictions.squeeze()
        labels = predictions.label_ids
        if logits.ndim > 1:
            logits = logits[:, 0]
        if np.any(np.isnan(logits)):
            logger.warning("NaN values found in predictions, replacing with 0")
            logits = np.nan_to_num(logits, nan=0.0)
        metrics = compute_metrics((logits, labels))
        logger.info(f"Test metrics ({args.model_name}): {metrics}")
        with open(os.path.join(args.model_path, f"res_{args.model_name.split('/')[-1]}.txt"), "a") as f:
            f.write(f"lr_{args.lr}_bs_{args.batch_size}_epochs_{args.num_train_epochs}: {metrics}\n")
        output_path = os.path.join("results", f"predictions_{args.model_name.split('/')[-1]}.txt")
        with open(output_path, "w") as f:
            for i, logit in enumerate(logits):
                pred = 1 if logit > 0 else 0
                f.write(f"{pred}\n")
                if i % 50 == 0:
                    logger.info(f"Written {i+1} predictions")
        logger.info(f"Predictions saved to {output_path}")
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        raise

def main(df, model_name="bert-base-uncased"):
    """Main function to run training and prediction for a given model"""
    global args
    args = parse_args()
    args.model_name = model_name

    try:
        wandb.init(project="empathy-classification", name=f"{model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}")
    except Exception as e:
        logger.warning(f"Failed to init W&B: {e}. Continuing without W&B.")
        args.report_to = None

    try:
        train_dataset, test_dataset, tokenizer, num_labels = load_and_prepare_data(args, df)
    except Exception as e:
        logger.error(f"Data preparation failed for {model_name}: {e}")
        raise

    if args.do_train:
        trainer = train_model(args, train_dataset, test_dataset, num_labels)

    if args.do_predict:
        if not args.do_train:
            try:
                model = AutoModelForSequenceClassification.from_pretrained(
                    os.path.join(args.model_path, args.model_name.split("/")[-1]), num_labels=num_labels
                )
            except Exception as e:
                logger.error(f"Failed to load model for prediction {args.model_name}: {e}")
                raise
            training_args = TrainingArguments(
                output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
                per_device_eval_batch_size=args.batch_size,
                fp16=True
            )
            trainer = CustomTrainer(
                model=model,
                args=training_args,
                compute_metrics=None
            )
        predict(trainer, test_dataset, args)

    wandb.finish()

if __name__ == "__main__":
    try:
        os.makedirs("results", exist_ok=True)
        df = pd.read_csv('chatbot_responses.csv', sep=',', on_bad_lines='skip', engine='python')
        logger.info(f"Raw CSV loaded with {len(df)} rows")
        data = df[['response', 'label', 'condition']]
        if data.empty:
            raise ValueError("Dataset is empty. Check input data.")
        logger.info(f"Filtered dataset with {len(data)} rows")
        models = [
            # "bert-base-uncased"
            # "bert-large-uncased"
            # "RoBERTa-base"
            # "DistilBERT-base-uncased"
        ]
        for model_name in models:
            logger.info(f"Training and evaluating {model_name}")
            main(data, model_name=model_name)
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise

Overwriting ex1.py


In [ ]:
os.makedirs("results", exist_ok=True)


In [ ]:
# save hrel as chatbot_responses.csv
import pandas as pd
hrel.to_csv('chatbot_responses.csv', index=False)

In [ ]:
# Bert-base-uncased


In [ ]:
#DistilBERT-base-uncased
!python ex1.py --do_train --do_predict --num_train_epochs 10 --lr 1e-5 --batch_size 16

2025-08-23 07:55:31.538551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755935731.598388   14711 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755935731.609107   14711 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755935731.641163   14711 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755935731.641240   14711 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755935731.641252   14711 computation_placer.cc:177] computation placer alr

In [ ]:
# RoBERTa-base
!python ex1.py --do_train --do_predict --num_train_epochs 10 --lr 1e-5 --batch_size 16

2025-08-23 07:38:56.012336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755934736.032990   10534 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755934736.039452   10534 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755934736.055329   10534 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755934736.055356   10534 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755934736.055360   10534 computation_placer.cc:177] computation placer alr

# Fine tuning with regression

In [ ]:
%%writefile regression.py


import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import wandb
import torch
import torch.nn as nn
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def parse_args():
    parser = argparse.ArgumentParser(description="Fine-tune models for binary empathy classification with weighted loss")
    parser.add_argument("--max_train_samples", type=int, default=-1, help="Number of training samples or -1 for all")
    parser.add_argument("--max_predict_samples", type=int, default=-1, help="Number of test samples or -1 for all")
    parser.add_argument("--num_train_epochs", type=int, default=3, help="Number of training epochs")
    parser.add_argument("--lr", type=float, default=2e-5, help="Learning rate")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for training and evaluation")
    parser.add_argument("--do_train", action="store_true", help="Run training")
    parser.add_argument("--do_predict", action="store_true", help="Run prediction")
    parser.add_argument("--model_path", type=str, default="./results", help="Path to save/load model")
    parser.add_argument("--model_name", type=str, default="bert-base-uncased", help="Model name (bert-base-uncased, roberta-base, distilbert-base-uncased, electra-small-discriminator)")
    return parser.parse_args()

def compute_sample_weights(labels):
    """Compute sample weights for binary classification based on class frequency"""
    bins = np.array([0, 1, 2])  # Bins for classes 0 and 1
    freq, _ = np.histogram(labels, bins=bins)
    freq = freq + 1e-6  # Avoid division by zero
    weights = np.max(freq) / freq * 2.0
    sample_weights = np.array([weights[int(label)] for label in labels])
    logger.info(f"Sample weights for classes [0, 1]: {weights}")
    return sample_weights

def undersample_data(df, high_label=1, low_label=0, ratio=1):
    """Undersample high-empathy class to achieve ~2:1 ratio with low-empathy class"""
    high_df = df[df['labels'] == high_label]
    low_df = df[df['labels'] == low_label]
    n_low = len(low_df)
    n_high = min(len(high_df), n_low * ratio)
    high_df = high_df.sample(n=n_high, random_state=42)
    balanced_df = pd.concat([high_df, low_df]).sample(frac=1, random_state=42).reset_index(drop=True)
    logger.info(f"After undersampling: {len(high_df)} high, {len(low_df)} low samples")
    return balanced_df

def plot_confusion_matrix(labels, predictions, phase="test", model_name="model"):
    """Plot and save confusion matrix"""
    cm = confusion_matrix(labels, predictions, labels=[0, 1])
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Low", "High"], yticklabels=["Low", "High"])
    plt.title(f"Confusion Matrix ({phase.capitalize()}, {model_name})")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    output_path = os.path.join("results", f"confusion_matrix_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"Confusion matrix saved to {output_path}")
    wandb.log({f"confusion_matrix_{phase}_{model_name}": wandb.Image(output_path)})
    return cm

def plot_f1_scores(labels, predictions, phase="test", model_name="model"):
    """Plot and save per-class F1 scores"""
    f1 = f1_score(labels, predictions, labels=[0, 1], average=None, zero_division=0)
    plt.figure(figsize=(6, 4))
    plt.bar(["Low", "High"], f1, color="skyblue")
    plt.title(f"F1 Scores per Class ({phase.capitalize()}, {model_name})")
    plt.xlabel("Class")
    plt.ylabel("F1 Score")
    plt.ylim(0, 1)
    output_path = os.path.join("results", f"f1_scores_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"F1 scores plot saved to {output_path}")
    wandb.log({f"f1_scores_{phase}_{model_name}": wandb.Image(output_path)})
    return f1

def load_and_prepare_data(args, df):
    """Load and preprocess data for binary classification"""
    df = df.dropna(subset=['response', 'label', 'condition'])
    df['label'] = df['label'].astype(float)
    if not ((df['label'] >= 0) & (df['label'] <= 9)).all():
        logger.warning("Some labels are outside [0, 9]. Clipping may affect results.")
    df['labels'] = (df['label'] >= 6).astype(int)
    logger.info(f"Label distribution (binary): {df['labels'].value_counts().to_string()}")
    wandb.log({"label_distribution": wandb.Histogram(df['labels'])})


    sample_weights = compute_sample_weights(df['labels'].values)
    df['weight'] = sample_weights


    df['input_text'] = df.apply(lambda x: f"condition_{int(x['condition'])}: {x['response']}", axis=1)

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    train_df = undersample_data(train_df, high_label=1, low_label=0, ratio=1)

    logger.info(f"Train size: {len(train_df)}, Test size: {len(test_df)}")

    train_dataset = Dataset.from_pandas(train_df[['input_text', 'labels', 'weight']])
    test_dataset = Dataset.from_pandas(test_df[['input_text', 'labels']])

    try:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    except Exception as e:
        logger.error(f"Failed to load tokenizer for {args.model_name}: {e}")
        raise

    def tokenize_function(examples):
        return tokenizer(
            examples["input_text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    if args.max_train_samples != -1:
        train_dataset = train_dataset.select(range(min(args.max_train_samples, len(train_dataset))))
    if args.max_predict_samples != -1:
        test_dataset = test_dataset.select(range(min(args.max_predict_samples, len(test_dataset))))

    train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "weight"])
    test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    return train_dataset, test_dataset, tokenizer, 1

def compute_metrics(eval_pred):
    """Compute accuracy, F1-score, and confusion matrix"""
    logits, labels = eval_pred
    logits = logits.squeeze()
    predictions = (logits > 0).astype(int)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    f1_per_class = f1_score(labels, predictions, labels=[0, 1], average=None, zero_division=0)
    cm = plot_confusion_matrix(labels, predictions, phase="test", model_name=args.model_name.split("/")[-1])
    f1_per_class_dict = {f"f1_class_{i}": float(f1_per_class[i]) for i in range(2)}
    metrics = {
        "accuracy": float(acc),
        "f1_macro": float(f1),
        **f1_per_class_dict
    }
    wandb.log(metrics)
    return metrics

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """Compute weighted BCE loss"""
        logger.debug(f"Inputs keys: {list(inputs.keys())}")
        labels = inputs.pop("labels")
        weights = inputs.pop("weight", torch.ones_like(labels))
        outputs = model(**inputs)
        logits = outputs.logits
        # Ensure logits and labels have compatible shapes
        if logits.dim() > 1:
            logits = logits.squeeze(-1)  # Squeeze only the last dimension if needed
        if logits.shape != labels.shape:
            logger.debug(f"Logits shape: {logits.shape}, Labels shape: {labels.shape}")
            logits = logits.view(-1)  # Flatten to [batch_size]
        loss_fn = nn.BCEWithLogitsLoss(reduction='none')
        loss = loss_fn(logits, labels.float())
        weighted_loss = torch.mean(weights * loss)
        return (weighted_loss, outputs) if return_outputs else weighted_loss

def train_model(args, train_dataset, test_dataset, num_labels):
    """Train the model with weighted loss"""
    os.makedirs(args.model_path, exist_ok=True)
    try:
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=num_labels)
    except Exception as e:
        logger.error(f"Failed to load model {args.model_name}: {e}")
        raise

    training_args = TrainingArguments(
        output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
        eval_strategy="no",
        save_strategy="epoch",
        learning_rate=args.lr,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.num_train_epochs,
        weight_decay=0.01,
        logging_steps=10,
        save_total_limit=2,
        report_to="wandb",
        run_name=f"{args.model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}",
        fp16=True,
        dataloader_drop_last=True  # Drop incomplete last batch
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=None,
        compute_metrics=None
    )

    try:
        trainer.train()
    except Exception as e:
        logger.error(f"Training failed for {args.model_name}: {e}")
        raise

    trainer.save_model(os.path.join(args.model_path, args.model_name.split("/")[-1]))
    return trainer

def predict(trainer, test_dataset, args):
    """Make predictions and compute metrics"""
    try:
        predictions = trainer.predict(test_dataset)
        logits = predictions.predictions.squeeze()
        labels = predictions.label_ids
        if logits.ndim > 1:
            logits = logits[:, 0]
        if np.any(np.isnan(logits)):
            logger.warning("NaN values found in predictions, replacing with 0")
            logits = np.nan_to_num(logits, nan=0.0)
        metrics = compute_metrics((logits, labels))
        logger.info(f"Test metrics ({args.model_name}): {metrics}")
        with open(os.path.join(args.model_path, f"res_{args.model_name.split('/')[-1]}.txt"), "a") as f:
            f.write(f"lr_{args.lr}_bs_{args.batch_size}_epochs_{args.num_train_epochs}: {metrics}\n")
        output_path = os.path.join("results", f"predictions_{args.model_name.split('/')[-1]}.txt")
        with open(output_path, "w") as f:
            for i, logit in enumerate(logits):
                pred = 1 if logit > 0 else 0
                f.write(f"{pred}\n")
                if i % 50 == 0:
                    logger.info(f"Written {i+1} predictions")
        logger.info(f"Predictions saved to {output_path}")
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        raise

def main(df, model_name="bert-base-uncased"):
    """Main function to run training and prediction for a given model"""
    global args
    args = parse_args()
    args.model_name = model_name

    try:
        wandb.init(project="empathy-classification", name=f"{model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}")
    except Exception as e:
        logger.warning(f"Failed to init W&B: {e}. Continuing without W&B.")
        args.report_to = None

    try:
        train_dataset, test_dataset, tokenizer, num_labels = load_and_prepare_data(args, df)
    except Exception as e:
        logger.error(f"Data preparation failed for {model_name}: {e}")
        raise

    if args.do_train:
        trainer = train_model(args, train_dataset, test_dataset, num_labels)

    if args.do_predict:
        if not args.do_train:
            try:
                model = AutoModelForSequenceClassification.from_pretrained(
                    os.path.join(args.model_path, args.model_name.split("/")[-1]), num_labels=num_labels
                )
            except Exception as e:
                logger.error(f"Failed to load model for prediction {args.model_name}: {e}")
                raise
            training_args = TrainingArguments(
                output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
                per_device_eval_batch_size=args.batch_size,
                fp16=True
            )
            trainer = CustomTrainer(
                model=model,
                args=training_args,
                compute_metrics=None
            )
        predict(trainer, test_dataset, args)

    wandb.finish()

if __name__ == "__main__":
    try:
        df = pd.read_csv('chatbot_responses.csv', sep=',', on_bad_lines='skip', engine='python')
        logger.info(f"Raw CSV loaded with {len(df)} rows")
        data = df[['response', 'label', 'condition']]
        if data.empty:
            raise ValueError("Dataset is empty. Check input data.")
        logger.info(f"Filtered dataset with {len(data)} rows")
        models = [
            "bert-base-uncased",
        ]
        for model_name in models:
            logger.info(f"Training and evaluating {model_name}")
            main(data, model_name=model_name)
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise

Overwriting regression.py


In [ ]:
%%writefile regression2.py
import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
import wandb
import torch
import torch.nn as nn
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def parse_args():
    parser = argparse.ArgumentParser(description="Evaluate roberta-base regression model with undersampling and confusion matrix")
    parser.add_argument("--max_train_samples", type=int, default=-1, help="Number of training samples or -1 for all")
    parser.add_argument("--max_predict_samples", type=int, default=-1, help="Number of test samples or -1 for all")
    parser.add_argument("--num_train_epochs", type=int, default=3, help="Number of training epochs")
    parser.add_argument("--lr", type=float, default=2e-5, help="Learning rate")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for training and evaluation")
    parser.add_argument("--do_train", action="store_true", help="Run training if model not found")
    parser.add_argument("--do_predict", action="store_true", help="Run prediction")
    parser.add_argument("--model_path", type=str, default="./results_regression", help="Path to save/load model")
    parser.add_argument("--model_name", type=str, default="roberta-base", help="Model name")
    return parser.parse_args()

def compute_sample_weights(labels):
    """Compute sample weights for regression based on label frequency"""
    bins = np.arange(0, 11)  # Bins for labels 0–9
    freq, _ = np.histogram(labels, bins=bins)
    freq = freq + 1e-6  # Avoid division by zero
    weights = np.max(freq) / freq * 2.0  # 2x amplification for rare classes
    sample_weights = np.array([weights[int(label)] for label in labels])
    logger.info(f"Sample weights for labels 0–9: {weights}")
    return sample_weights

def undersample_data(df, majority_labels=[7, 8, 9], ratio=2):
    """Undersample majority labels (7–9) to ~3:1 ratio with the rarest label"""
    label_counts = df['labels'].value_counts().to_dict()
    minority_counts = {k: v for k, v in label_counts.items() if k < 7}

    min_count = min(minority_counts.values())
    target_count = int(min_count * ratio)
    logger.info(f"Rarest label count: {min_count}, Target count for majority labels: {target_count}")

    dfs = []
    for label in range(10):
        label_df = df[df['labels'] == label]
        if label in majority_labels and len(label_df) > target_count:
            label_df = label_df.sample(n=target_count, random_state=42)
            logger.info(f"Undersampled label {label} from {label_counts.get(label, 0)} to {target_count}")
        dfs.append(label_df)

    balanced_df = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)
    logger.info(f"After undersampling: {len(balanced_df)} samples, Label distribution: {balanced_df['labels'].value_counts().sort_index().to_string()}")
    return balanced_df

def plot_mae_per_score(labels, predictions, phase="test", model_name="model"):
    """Plot and save MAE per score (0–9) with sample counts"""
    mae_per_score = []
    sample_counts = []
    for score in range(10):
        mask = labels == score
        count = np.sum(mask)
        sample_counts.append(count)
        if count > 0:
            mae = mean_absolute_error(labels[mask], predictions[mask])
        else:
            mae = 0.0
        mae_per_score.append(mae)
    plt.figure(figsize=(8, 5))
    plt.bar(range(10), mae_per_score, color="skyblue")
    plt.title(f"MAE per Empathy Score ({phase.capitalize()}, {model_name})")
    plt.xlabel("Empathy Score")
    plt.ylabel("Mean Absolute Error")
    plt.ylim(0, max(mae_per_score + [1]) * 1.2)
    for i, (mae, count) in enumerate(zip(mae_per_score, sample_counts)):
        plt.text(i, mae + 0.05, f"n={count}", ha="center", fontsize=8)
    output_path = os.path.join("results_regression", f"mae_per_score_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"MAE per score plot saved to {output_path}")
    wandb.log({f"mae_per_score_{phase}_{model_name}": wandb.Image(output_path)})
    return mae_per_score, sample_counts

def plot_confusion_matrix(labels, predictions, phase="test", model_name="model"):
    """Plot and save confusion matrix for rounded predictions"""
    cm = confusion_matrix(labels, predictions, labels=range(10))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
    plt.title(f"Confusion Matrix ({phase.capitalize()}, {model_name})")
    plt.xlabel("Predicted Score")
    plt.ylabel("True Score")
    output_path = os.path.join("results_regression", f"confusion_matrix_{phase}_{model_name}.png")
    plt.savefig(output_path)
    plt.close()
    logger.info(f"Confusion matrix saved to {output_path}")
    wandb.log({f"confusion_matrix_{phase}_{model_name}": wandb.Image(output_path)})
    return cm

def load_and_prepare_data(args, df):
    """Load and preprocess data for regression with undersampling"""
    df = df.dropna(subset=['response', 'label', 'condition'])
    df['labels'] = df['label'].astype(float)
    if not ((df['labels'] >= 0) & (df['labels'] <= 9)).all():
        logger.warning("Some labels are outside [0, 9]. Clipping may affect results.")
    logger.info(f"Original label distribution: {df['labels'].value_counts().sort_index().to_string()}")


    df['input_text'] = df.apply(lambda x: f"condition_{int(x['condition'])}: {x['response']}", axis=1)

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    train_df = undersample_data(train_df, majority_labels=[7, 8, 9], ratio=2)

    sample_weights = compute_sample_weights(train_df['labels'].values)
    train_df['weight'] = sample_weights

    logger.info(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
    logger.info(f"Test label distribution: {test_df['labels'].value_counts().sort_index().to_string()}")

    train_dataset = Dataset.from_pandas(train_df[['input_text', 'labels', 'weight']])
    test_dataset = Dataset.from_pandas(test_df[['input_text', 'labels']])

    try:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    except Exception as e:
        logger.error(f"Failed to load tokenizer for {args.model_name}: {e}")
        raise

    def tokenize_function(examples):
        return tokenizer(
            examples["input_text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    if args.max_train_samples != -1:
        train_dataset = train_dataset.select(range(min(args.max_train_samples, len(train_dataset))))
    if args.max_predict_samples != -1:
        test_dataset = test_dataset.select(range(min(args.max_predict_samples, len(test_dataset))))

    train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "weight"])
    test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    return train_dataset, test_dataset, tokenizer, 1

def compute_metrics(eval_pred):
    """Compute MSE, MAE, per-score MAE, and confusion matrix"""
    global args
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    if np.any(np.isnan(predictions)):
        logger.warning("NaN values found in predictions, replacing with 0")
        predictions = np.nan_to_num(predictions, nan=0.0)
    pred_scores_rounded = np.round(predictions).clip(0, 9).astype(int)
    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    mae_per_score, sample_counts = plot_mae_per_score(labels, predictions, phase="test", model_name=args.model_name.split("/")[-1])
    cm = plot_confusion_matrix(labels, pred_scores_rounded, phase="test", model_name=args.model_name.split("/")[-1])
    metrics = {
        "mse": float(mse),
        "mae": float(mae),
        **{f"mae_score_{i}": float(mae_per_score[i]) for i in range(10)},
        **{f"sample_count_score_{i}": int(sample_counts[i]) for i in range(10)}
    }
    wandb.log(metrics)
    return metrics

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """Compute weighted MSE loss"""
        logger.debug(f"Inputs keys: {list(inputs.keys())}")
        labels = inputs.pop("labels")
        weights = inputs.pop("weight", torch.ones_like(labels))
        outputs = model(**inputs)
        predictions = outputs.logits.squeeze()
        loss_fn = nn.MSELoss(reduction='none')
        loss = loss_fn(predictions, labels.float())
        weighted_loss = torch.mean(weights * loss)
        return (weighted_loss, outputs) if return_outputs else weighted_loss

def train_model(args, train_dataset, test_dataset, num_labels):
    """Train the model with weighted MSE loss"""
    os.makedirs(args.model_path, exist_ok=True)
    try:
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=num_labels)
    except Exception as e:
        logger.error(f"Failed to load model {args.model_name}: {e}")
        raise

    training_args = TrainingArguments(
        output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
        eval_strategy="no",
        save_strategy="epoch",
        learning_rate=args.lr,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.num_train_epochs,
        weight_decay=0.01,
        logging_steps=10,
        save_total_limit=2,
        report_to="wandb",
        run_name=f"{args.model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}_undersampled",
        fp16=True
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=None,
        compute_metrics=None
    )

    try:
        trainer.train()
    except Exception as e:
        logger.error(f"Training failed for {args.model_name}: {e}")
        raise

    trainer.save_model(os.path.join(args.model_path, args.model_name.split("/")[-1]))
    return trainer

def predict(trainer, test_dataset, args):
    """Make predictions and compute metrics"""
    try:
        predictions = trainer.predict(test_dataset)
        pred_scores = predictions.predictions.squeeze()
        labels = predictions.label_ids
        if pred_scores.ndim > 1:
            pred_scores = pred_scores[:, 0]
        if np.any(np.isnan(pred_scores)):
            logger.warning("NaN values found in predictions, replacing with 0")
            pred_scores = np.nan_to_num(pred_scores, nan=0.0)
        pred_scores_rounded = np.round(pred_scores).clip(0, 9).astype(int)
        metrics = compute_metrics((pred_scores, labels))
        logger.info(f"Test metrics ({args.model_name}): {metrics}")
        with open(os.path.join(args.model_path, f"res_{args.model_name.split('/')[-1]}_undersampled.txt"), "a") as f:
            f.write(f"lr_{args.lr}_bs_{args.batch_size}_epochs_{args.num_train_epochs}: {metrics}\n")
        output_path = os.path.join("results_regression", f"predictions_{args.model_name.split('/')[-1]}_undersampled.txt")
        with open(output_path, "w") as f:
            for i, (score, rounded_score) in enumerate(zip(pred_scores, pred_scores_rounded)):
                f.write(f"Raw: {score:.4f}, Rounded: {int(rounded_score)}\n")
                if i % 50 == 0:
                    logger.info(f"Written {i+1} predictions")
        logger.info(f"Predictions saved to {output_path}")
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        raise

def main(df, model_name="bert-base-uncased"):
    """Main function to run training and prediction with undersampling and confusion matrix"""
    global args
    args = parse_args()
    args.model_name = model_name

    try:
        wandb.init(project="empathy-regression-undersample-cm", name=f"{model_name.split('/')[-1]}_lr_{args.lr}_bs_{args.batch_size}_undersampled")
    except Exception as e:
        logger.warning(f"Failed to init W&B: {e}. Continuing without W&B.")
        args.report_to = None

    try:
        train_dataset, test_dataset, tokenizer, num_labels = load_and_prepare_data(args, df)
    except Exception as e:
        logger.error(f"Data preparation failed for {model_name}: {e}")
        raise

    if args.do_train or not os.path.exists(os.path.join(args.model_path, args.model_name.split("/")[-1])):
        logger.info(f"No trained model found at {args.model_path}/{args.model_name.split('/')[-1]}. Training new model.")
        trainer = train_model(args, train_dataset, test_dataset, num_labels)
    else:
        try:
            model = AutoModelForSequenceClassification.from_pretrained(
                os.path.join(args.model_path, args.model_name.split("/")[-1]), num_labels=num_labels
            )
        except Exception as e:
            logger.error(f"Failed to load model for prediction {args.model_name}: {e}")
            raise
        training_args = TrainingArguments(
            output_dir=os.path.join(args.model_path, args.model_name.split("/")[-1]),
            per_device_eval_batch_size=args.batch_size,
            fp16=True
        )
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            compute_metrics=None
        )

    if args.do_predict:
        predict(trainer, test_dataset, args)

    wandb.finish()

if __name__ == "__main__":
    try:
        df = pd.read_csv('chatbot_responses.csv', sep=',', on_bad_lines='skip', engine='python')
        logger.info(f"Raw CSV loaded with {len(df)} rows")
        data = df[['response', 'label', 'condition']]
        if data.empty:
            raise ValueError("Dataset is empty. Check input data.")
        logger.info(f"Filtered dataset with {len(data)} rows")
        main(data, model_name="bert-base-uncased")
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise


Overwriting regression2.py


In [ ]:
!python regression2.py --do_train --do_predict --num_train_epochs 7 --lr 1e-5 --batch_size 16

2025-06-28 09:10:24.327468: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-28 09:10:24.346305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751101824.370053   43705 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751101824.377129   43705 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 09:10:24.399672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr